In [ ]:

!pip install pandas
!pip install tqdm
!pip install matplotlib
!pip install sklearn
!pip install transformers

# Sentiment Analysis with Deep Learning using BERT

### Prerequisites

- Intermediate-level knowledge of Python 3 (NumPy and Pandas preferably, but not required)
- Exposure to PyTorch usage
- Basic understanding of Deep Learning and Language Models (BERT specifically)

### Project Outline

**Task 1**: Introduction (this section)

**Task 2**: Exploratory Data Analysis and Preprocessing

**Task 3**: Training/Validation Split

**Task 4**: Loading Tokenizer and Encoding our Data

**Task 5**: Setting up BERT Pretrained Model

**Task 6**: Creating Data Loaders

**Task 7**: Setting Up Optimizer and Scheduler

**Task 8**: Defining our Performance Metrics

**Task 9**: Creating our Training Loop

**Task 10**: Loading and Evaluating our Model

## Task 1: Introduction

### What is BERT

BERT is a large-scale transformer-based Language Model that can be finetuned for a variety of tasks.

For more information, the original paper can be found [here](https://arxiv.org/abs/1810.04805). 

[HuggingFace documentation](https://huggingface.co/transformers/model_doc/bert.html)

[Bert documentation](https://characters.fandom.com/wiki/Bert_(Sesame_Street) ;)

<img src="Images/BERT_diagrams.png" width="1000">

## Task 2: Exploratory Data Analysis and Preprocessing

We will use the SMILE Twitter dataset.

_Wang, Bo; Tsakalidis, Adam; Liakata, Maria; Zubiaga, Arkaitz; Procter, Rob; Jensen, Eric (2016): SMILE Twitter Emotion dataset. figshare. Dataset. https://doi.org/10.6084/m9.figshare.3187909.v2_

In [ ]:
import torch
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("./data/smile-annotations-final.csv", names= ["id","text","category"])
data.set_index("id", inplace=True)
data.head()

In [ ]:
data.category.value_counts().plot(kind="bar")

In [ ]:
data_cleaned = data.loc[
    ~(data["category"].str.contains('[|]')
    | data["category"].str.contains('(nocode)'))
    ]

data_cleaned.category.value_counts().plot(kind="bar")

In [ ]:
possible_labels = data_cleaned.category.unique()
labels_dict = {}
for index, label in enumerate(possible_labels):
    labels_dict[label] = index

In [ ]:
data_cleaned["label"] = data_cleaned.category.map(labels_dict)

## Task 3: Training/Validation Split

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
df_train, df_test, y_train, y_test = train_test_split(
    data_cleaned.drop(["label","category"], axis=1), 
    data_cleaned["label"], 
    test_size=0.15, 
    random_state=42, 
    stratify=data_cleaned["label"]
)


In [ ]:
data_cleaned["data_type"] = "not_set"
data_cleaned.loc[df_train.index, "data_type"] = "train"
data_cleaned.loc[df_test.index, "data_type"] = "val"


In [ ]:
data_cleaned.groupby(["category","label","data_type"]).count()

## Task 4: Loading Tokenizer and Encoding our Data

In [ ]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [ ]:
tokenizer = BertTokenizer.from_pretrained(
    "bert-base-uncased",
    do_lower_case = True
)

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    data_cleaned[data_cleaned.data_type=="train"].text.values,
    add_special_tokens=True, # add special tokens such as [EOS,CLS]
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors="pt"

)

encoded_data_val = tokenizer.batch_encode_plus(
    data_cleaned[data_cleaned.data_type=="val"].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors="pt"

)

input_ids_train = encoded_data_train["input_ids"]
attention_masks_train = encoded_data_train["attention_mask"]
labels_train = torch.tensor(data_cleaned[data_cleaned.data_type=="train"].label.values)

input_ids_val = encoded_data_val["input_ids"]
attention_masks_val = encoded_data_val["attention_mask"]
labels_val = torch.tensor(data_cleaned[data_cleaned.data_type=="val"].label.values)

In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)

dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

## Task 5: Setting up BERT Pretrained Model

In [ ]:
from transformers import BertForSequenceClassification

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels= len(labels_dict),
    output_attentions=False,
    output_hidden_states=False
)

## Task 6: Creating Data Loaders

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [ ]:
batch_size = 4

data_loader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

data_loader_val = DataLoader(
    dataset_val,
    sampler=SequentialSampler(dataset_val),
    batch_size=batch_size
)

## Task 7: Setting Up Optimizer and Scheduler

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [ ]:
optimizer = AdamW(
    model.parameters(),
    lr=1e-5, #2e-5<5e-5
    eps=1e-8
)

In [ ]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = len(data_loader_train)*epochs
)

## Task 8: Defining our Performance Metrics

Accuracy metric approach originally used in accuracy function in [this tutorial](https://mccormickml.com/2019/07/22/BERT-fine-tuning/#41-bertforsequenceclassification).

In [ ]:
import numpy as np

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return f1_score(labels_flat, preds_flat, average="weighted")

In [ ]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k,v in labels_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[preds_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f"Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n")


## Task 9: Creating our Training Loop

Approach adapted from an older version of HuggingFace's `run_glue.py` script. Accessible [here](https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128).

In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device("cuda")
model.to(device)
print("ok")

In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals


In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()

    loss_train_total = 0

    progress_bar = tqdm(
        data_loader_train, 
        desc=f"Epoch: {epoch}",
        leave=False,
        disable=False
        )
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)

        inputs = {
            "input_ids"     : batch[0],
            "attention_mask": batch[1],
            "labels"        : batch[2]

        }

        outputs = model(**inputs)

        loss = outputs[0]
        loss_train_total += float(loss.item())
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({"training_loss": "{:.3f}".format(loss.item()/len(batch))})
    torch.save(model.state_dict(), f"Models/BERT_ft_epoch{epoch}.model")
    tqdm.write(f"\nEpoch {epoch}")

    loss_train_avg = loss_train_total/len(data_loader_train)

    tqdm.write(f"Training loss: {loss_train_avg}")

    val_loss, predictions, true_vals = evaluate(data_loader_val)
    val_f1 = f1_score_func(predictions, true_vals)

    tqdm.write(f"Valisation loss: {val_loss}")
    tqdm.write(f"F1 score: {val_f1}")

        

## Task 10: Loading and Evaluating our Model

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(labels_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

In [ ]:
model.load_state_dict(
    torch.load("Models/BERT_ft_epoch9.model" , map_location=torch.device("cuda"))
    
)

In [ ]:
_, predictions, true_vals = evaluate(data_loader_val)

In [ ]:
accuracy_per_class(predictions, true_vals)